In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# ================================
# 1. Definición de Parámetros
# ================================

intervalo_minutos = 10
hora_inicio = 5 * 60  # 5 am en minutos
hora_fin = 21 * 60    # 9 pm en minutos

# Demanda predicha para todas las rutas y sentidos
demanda_predicha = {
    'PRADERA_27_sentido_1': [36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 46, 47, 48, 49, 49, 49, 50,
                               50, 50, 49, 49, 49, 49, 49, 48, 48, 48, 48, 47, 47, 46, 46, 45, 45,
                               45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45,
                               45, 45, 45, 45, 45, 45, 45, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46,
                               46, 46, 46, 46, 47, 46, 46, 45, 44, 44, 43, 42, 41, 40, 39, 38,
                               37, 35, 34, 33, 32, 31, 29, 28, 27, 26, 26, 25],
    'PRADERA_27_sentido_2': [36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 46, 47, 48, 49, 49, 49, 50,
                               50, 50, 49, 49, 49, 49, 49, 48, 48, 48, 48, 47, 47, 46, 46, 45, 45,
                               45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45,
                               45, 45, 45, 45, 45, 45, 45, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46,
                               46, 46, 46, 46, 47, 46, 46, 45, 44, 44, 43, 42, 41, 40, 39, 38,
                               37, 35, 34, 33, 32, 31, 29, 28, 27, 26, 26, 25],
    'PANZENU_sentido_1': [21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 35, 36, 36,
                           36, 37, 37, 37, 37, 37, 37, 37, 37, 37, 36, 36, 36, 34, 34, 34, 34,
                           34, 34, 34, 33, 33, 33, 33, 33, 33, 33, 33, 33, 32, 32, 32, 32, 32,
                           32, 32, 33, 33, 33, 33, 33, 34, 34, 34, 34, 35, 35, 35, 35, 36, 36,
                           36, 36, 36, 35, 37, 36, 36, 35, 34, 33, 32, 31, 30, 29, 27, 26, 24,
                           23, 21, 20, 19, 17, 14, 13, 12, 11, 10,  9],
    'PANZENU_sentido_2': [21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 35, 36, 36,
                           36, 37, 37, 37, 37, 37, 37, 37, 37, 37, 36, 36, 36, 34, 34, 34, 34,
                           34, 34, 34, 33, 33, 33, 33, 33, 33, 33, 33, 33, 32, 32, 32, 32, 32,
                           32, 32, 33, 33, 33, 33, 33, 34, 34, 34, 34, 35, 35, 35, 35, 36, 36,
                           36, 36, 36, 35, 37, 36, 36, 35, 34, 33, 32, 31, 30, 29, 27, 26, 24,
                           23, 21, 20, 19, 17, 14, 13, 12, 11, 10,  9],
    'SANTANDER_sentido_1': [23, 23, 24, 25, 26, 26, 27, 28, 28, 29, 29, 29, 31, 31, 31, 31, 31,
                             31, 31, 31, 31, 31, 31, 31, 31, 30, 30, 30, 30, 30, 29, 29, 29, 29,
                             29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
                             29, 29, 29, 30, 30, 30, 30, 30, 30, 30, 30, 30, 31, 31, 31, 31, 31,
                             31, 31, 31, 31, 32, 31, 31, 31, 30, 30, 29, 28, 28, 27, 26, 25, 24,
                             23, 22, 22, 21, 20, 18, 17, 17, 16, 16, 15],
    'SANTANDER_sentido_2': [23, 23, 24, 25, 26, 26, 27, 28, 28, 29, 29, 29, 31, 31, 31, 31, 31,
                             31, 31, 31, 31, 31, 31, 31, 31, 30, 30, 30, 30, 30, 29, 29, 29, 29,
                             29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
                             29, 29, 29, 30, 30, 30, 30, 30, 30, 30, 30, 30, 31, 31, 31, 31, 31,
                             31, 31, 31, 31, 32, 31, 31, 31, 30, 30, 29, 28, 28, 27, 26, 25, 24,
                             23, 22, 22, 21, 20, 18, 17, 17, 16, 16, 15],
    'MOGAMBO_22_sentido_1': [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 21, 22, 23, 23, 24, 24, 24,
                               24, 24, 24, 24, 24, 24, 24, 24, 23, 23, 23, 23, 23, 22, 22, 21, 21,
                               21, 21, 21, 21, 21, 21, 21, 21, 21, 20, 20, 20, 20, 20, 20, 20, 20,
                               20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21,
                               21, 21, 21, 21, 21, 21, 20, 20, 19, 18, 18, 17, 16, 15, 14, 13, 12,
                               11, 10,  9,  8,  7,  5,  4,  4,  3,  2,  2],
    'MOGAMBO_22_sentido_2': [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 21, 22, 23, 23, 24, 24, 24,
                               24, 24, 24, 24, 24, 24, 24, 24, 23, 23, 23, 23, 23, 22, 22, 21, 21,
                               21, 21, 21, 21, 21, 21, 21, 21, 21, 20, 20, 20, 20, 20, 20, 20, 20,
                               20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21,
                               21, 21, 21, 21, 21, 21, 20, 20, 19, 18, 18, 17, 16, 15, 14, 13, 12,
                               11, 10,  9,  8,  7,  5,  4,  4,  3,  2,  2],
    'TAMBO_CIRCUNVALAR_sentido_1': [10, 10, 11, 11, 12, 12, 13, 13, 13, 14, 14, 14, 16, 16, 16, 16, 16,
                                     16, 16, 16, 16, 16, 16, 15, 15, 15, 15, 15, 15, 15, 14, 14, 14, 14,
                                     14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 14, 14,
                                     14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15,
                                     15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 15, 15, 15, 14, 14, 13, 13,
                                     12, 11, 11, 10, 10,  8,  7,  6,  6,  6,  5],
    'TAMBO_CIRCUNVALAR_sentido_2': [10, 10, 11, 11, 12, 12, 13, 13, 13, 14, 14, 14, 16, 16, 16, 16, 16,
                                     16, 16, 16, 16, 16, 16, 15, 15, 15, 15, 15, 15, 15, 14, 14, 14, 14,
                                     14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 14, 14,
                                     14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15,
                                     15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 15, 15, 15, 14, 14, 13, 13,
                                     12, 11, 11, 10, 10,  8,  7,  6,  6,  6,  5],
    'KM30_sentido_1': [3,  3,  4,  4,  5,  6,  6,  7,  7,  8,  9,  9,  7,  8,  8,  8,  9,
                        9,  9,  9,  9,  9,  9,  9,  9,  9,  8,  8,  8,  8, 10,  9,  9,  9,
                        8,  8,  8,  8,  8,  8,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,
                        8,  8,  8,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10,  9,
                        9,  9,  9,  9,  6,  6,  5,  5,  4,  4,  3,  3,  2,  2,  1,  1,  0,
                        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    'KM30_sentido_2': [3,  3,  4,  4,  5,  6,  6,  7,  7,  8,  9,  9,  7,  8,  8,  8,  9,
                        9,  9,  9,  9,  9,  9,  9,  9,  9,  8,  8,  8,  8, 10,  9,  9,  9,
                        8,  8,  8,  8,  8,  8,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,
                        8,  8,  8,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10,  9,
                        9,  9,  9,  9,  6,  6,  5,  5,  4,  4,  3,  3,  2,  2,  1,  1,  0,
                        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    'DORADO_sentido_1': [11, 11, 11, 12, 12, 13, 13, 14, 14, 14, 15, 15, 16, 16, 17, 17, 17,
                           17, 17, 17, 17, 17, 17, 16, 16, 16, 16, 16, 16, 16, 15, 15, 14, 14,
                           14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
                           15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
                           15, 15, 15, 14, 15, 15, 15, 15, 14, 14, 14, 13, 13, 12, 12, 12, 11,
                           11, 10, 10,  9,  9,  7,  7,  7,  7,  6,  6],
    'DORADO_sentido_2': [11, 11, 11, 12, 12, 13, 13, 14, 14, 14, 15, 15, 16, 16, 17, 17, 17,
                           17, 17, 17, 17, 17, 17, 16, 16, 16, 16, 16, 16, 16, 15, 15, 14, 14,
                           14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
                           15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
                           15, 15, 15, 14, 15, 15, 15, 15, 14, 14, 14, 13, 13, 12, 12, 12, 11,
                           11, 10, 10,  9,  9,  7,  7,  7,  7,  6,  6],
    'KM 15_sentido_1': [2, 3, 3, 3, 4, 4, 4, 5, 5, 6, 6, 6, 5, 6, 6, 6, 6, 7, 7, 7, 7, 7,
                        7, 7, 7, 7, 7, 7, 7, 6, 7, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6,
                        6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 6,
                        6, 6, 6, 6, 6, 5, 4, 4, 3, 3, 3, 3, 2, 2, 2, 2, 1, 1, 1, 1, 0, 0,
                        0, 0, 1, 1, 1, 0, 0, 0],
    'KM 15_sentido_2': [2, 3, 3, 3, 4, 4, 4, 5, 5, 6, 6, 6, 5, 6, 6, 6, 6, 7, 7, 7, 7, 7,
                        7, 7, 7, 7, 7, 7, 7, 6, 7, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6,
                        6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 6,
                        6, 6, 6, 6, 6, 5, 4, 4, 3, 3, 3, 3, 2, 2, 2, 2, 1, 1, 1, 1, 0, 0,
                        0, 0, 1, 1, 1, 0, 0, 0]
}

# Definición de rutas y sus parámetros
rutas = {
    'PRADERA_27': {
        'sentidos': {1: [], 2: []},  # Se llenarán más adelante
        'B': 8,  # Número de buses disponibles para esta ruta
        'C': 52, # Capacidad de cada bus
        'T_l_lprime': {
            ('P', 'C1'): 15,  
            ('P', 'C2'): 15,  
            ('C1', 'P'): 15,
            ('C2', 'P'): 15,
            ('C1', 'C2'): 41, 
            ('C2', 'C1'): 39,
            ('C1', 'C1'): 10,  
            ('C2', 'C2'): 10,
            ('P', 'P'): 0     # Permanencia en el patio: sin costo
        },
        'c_mov': 10,        # Costo fijo por movimiento
        'c_bus': 5,         # Costo por mantener el bus en operación
        'c_ingreso': 2,     # Ingreso por pasajero transportado
        'c_espera': 5       # Costo de permanencia en cabeceras
    },
    'PANZENU': {
        'sentidos': {1: [], 2: []},
        'B': 7,
        'C': 52,
        'T_l_lprime': {
            ('P', 'C1'): 15,  
            ('P', 'C2'): 15,  
            ('C1', 'P'): 15,
            ('C2', 'P'): 15,
            ('C1', 'C2'): 39,
            ('C2', 'C1'): 36,
            ('C1', 'C1'): 10,
            ('C2', 'C2'): 10,
            ('P', 'P'): 0
        },
        'c_mov': 10,        # Costo fijo por movimiento
        'c_bus': 5,         # Costo por mantener el bus en operación
        'c_ingreso': 2,     # Ingreso por pasajero transportado
        'c_espera': 5       # Costo de permanencia en cabeceras
    },
    'SANTANDER': {
        'sentidos': {1: [], 2: []},
        'B': 7,
        'C': 52,
        'T_l_lprime': {
            ('P', 'C1'): 15,  
            ('P', 'C2'): 15,  
            ('C1', 'P'): 15,
            ('C2', 'P'): 15,
            ('C1', 'C2'): 49,
            ('C2', 'C1'): 47,
            ('C1', 'C1'): 10,
            ('C2', 'C2'): 10,
            ('P', 'P'): 0
        },
        'c_mov': 10,        # Costo fijo por movimiento
        'c_bus': 5,         # Costo por mantener el bus en operación
        'c_ingreso': 2,     # Ingreso por pasajero transportado
        'c_espera': 5       # Costo de permanencia en cabeceras
    },
    'MOGAMBO_22': {
        'sentidos': {1: [], 2: []},
        'B': 7,
        'C': 52,
        'T_l_lprime': {
            ('P', 'C1'): 15,  
            ('P', 'C2'): 15,  
            ('C1', 'P'): 15,
            ('C2', 'P'): 15,
            ('C1', 'C2'): 28,
            ('C2', 'C1'): 28,
            ('C1', 'C1'): 29,
            ('C2', 'C2'): 29,
            ('P', 'P'): 0
        },
        'c_mov': 10,        # Costo fijo por movimiento
        'c_bus': 5,         # Costo por mantener el bus en operación
        'c_ingreso': 2,     # Ingreso por pasajero transportado
        'c_espera': 5       # Costo de permanencia en cabeceras
    },
    'TAMBO_CIRCUNVALAR': {
        'sentidos': {1: [], 2: []},
        'B': 5,
        'C': 52,
        'T_l_lprime': {
            ('P', 'C1'): 15,  
            ('P', 'C2'): 15,  
            ('C1', 'P'): 15,
            ('C2', 'P'): 15,
            ('C1', 'C2'): 39,
            ('C2', 'C1'): 34,
            ('C1', 'C1'): 10,
            ('C2', 'C2'): 10,
            ('P', 'P'): 0
        },
        'c_mov': 10,        # Costo fijo por movimiento
        'c_bus': 5,         # Costo por mantener el bus en operación
        'c_ingreso': 2,     # Ingreso por pasajero transportado
        'c_espera': 5       # Costo de permanencia en cabeceras
    },
    'KM30': {
        'sentidos': {1: [], 2: []},
        'B': 1,
        'C': 52,
        'T_l_lprime': {
            ('P', 'C1'): 15,  
            ('P', 'C2'): 15,  
            ('C1', 'P'): 15,
            ('C2', 'P'): 15,
            ('C1', 'C2'): 40,
            ('C2', 'C1'): 34,
            ('C1', 'C1'): 10,
            ('C2', 'C2'): 10,
            ('P', 'P'): 0
        },
        'c_mov': 10,        # Costo fijo por movimiento
        'c_bus': 5,         # Costo por mantener el bus en operación
        'c_ingreso': 2,     # Ingreso por pasajero transportado
        'c_espera': 5       # Costo de permanencia en cabeceras
    },
    'DORADO': {
        'sentidos': {1: [], 2: []},
        'B': 7,
        'C': 52,
        'T_l_lprime': {
            ('P', 'C1'): 15,  
            ('P', 'C2'): 15,  
            ('C1', 'P'): 15,
            ('C2', 'P'): 15,
            ('C1', 'C2'): 30,
            ('C2', 'C1'): 30,
            ('C1', 'C1'): 10,
            ('C2', 'C2'): 10,
            ('P', 'P'): 0
        },
        'c_mov': 10,        # Costo fijo por movimiento
        'c_bus': 5,         # Costo por mantener el bus en operación
        'c_ingreso': 2,     # Ingreso por pasajero transportado
        'c_espera': 5       # Costo de permanencia en cabeceras
    },
    'KM 15': {
        'sentidos': {1: [], 2: []},
        'B': 2,
        'C': 52,
        'T_l_lprime': {
            ('P', 'C1'): 15,  
            ('P', 'C2'): 15,  
            ('C1', 'P'): 15,
            ('C2', 'P'): 15,
            ('C1', 'C2'): 24,
            ('C2', 'C1'): 22,
            ('C1', 'C1'): 10,
            ('C2', 'C2'): 10,
            ('P', 'P'): 0
        },
        'c_mov': 10,        # Costo fijo por movimiento
        'c_bus': 5,         # Costo por mantener el bus en operación
        'c_ingreso': 2,     # Ingreso por pasajero transportado
        'c_espera': 5       # Costo de permanencia en cabeceras
    }
}

# Función para ajustar tiempos de viaje al número de intervalos
def ajustar_tiempos(tiempos, intervalo_minutos=10):
    tiempos_ajustados = {}
    for (origen, destino), tiempo in tiempos.items():
        tiempos_ajustados[(origen, destino)] = int(np.ceil(tiempo / intervalo_minutos))
    return tiempos_ajustados

# Ajustar tiempos de viaje para cada ruta
for ruta, parametros in rutas.items():
    rutas[ruta]['T_l_lprime'] = ajustar_tiempos(parametros['T_l_lprime'], intervalo_minutos)

# Asignar las demandas predichas a cada ruta y sentido
for ruta in rutas.keys():
    for s in [1, 2]:
        key = f"{ruta}_sentido_{s}"
        rutas[ruta]['sentidos'][s] = demanda_predicha[key]


In [3]:
def optimizar_ruta(nombre_ruta, parametros_ruta, intervalo_minutos=10, hora_inicio=5*60, hora_fin=21*60,
                  mipgap=0.005, timelimit=3600, mipsepcuts=2, impliedcuts=1, networkcuts=1,
                  flowcovercuts=1, liftprojectcuts=2, cliquecuts=2):
    """
    Optimiza el despacho de buses para una ruta específica.

    Args:
        nombre_ruta (str): Nombre de la ruta.
        parametros_ruta (dict): Parámetros específicos de la ruta.
        intervalo_minutos (int, optional): Duración de cada intervalo de tiempo en minutos. Default es 10.
        hora_inicio (int, optional): Hora de inicio en minutos desde medianoche. Default es 5*60 (5 am).
        hora_fin (int, optional): Hora de fin en minutos desde medianoche. Default es 21*60 (9 pm).
        threads (int, optional): Número de hilos para Gurobi. Si es None, usa el valor por defecto.
        heuristics (float, optional): Nivel de heurísticas para Gurobi. Si es None, usa el valor por defecto.
        presolve (int, optional): Nivel de presolve para Gurobi. Si es None, usa el valor por defecto.
        mipfocus (int, optional): Enfoque del MIP para Gurobi. Si es None, usa el valor por defecto.
        mipgap (float, optional): Brecha de MIP permitida. Default es 0.01 (1%).
        timelimit (int, optional): Límite de tiempo en segundos para Gurobi. Default es 3600 (1 hora).

    Returns:
        dict: Resultados de la optimización.
    """
    # Extraer parámetros de la ruta
    S = [1, 2]  # Sentidos de servicio
    D = parametros_ruta['sentidos']  # Demanda por sentido y tiempo
    B = parametros_ruta['B']        # Número de buses
    C = parametros_ruta['C']        # Capacidad de cada bus
    T_l_lprime = parametros_ruta['T_l_lprime']  # Tiempos de viaje en intervalos
    c_mov = parametros_ruta['c_mov']
    c_bus = parametros_ruta['c_bus']
    c_ingreso = parametros_ruta['c_ingreso']
    c_espera = parametros_ruta['c_espera']

    # Definir intervalos de tiempo
    N = int((hora_fin - hora_inicio) / intervalo_minutos)  # Número de intervalos
    T = list(range(N+1))  # 0,1,...,N

    # Ubicaciones (suponiendo las mismas para todas las rutas)
    L = ['P', 'C1', 'C2']

    # Definir conjuntos de buses
    K = list(range(1, B+1))  # 1,2,...,B

    # Definir arcos y parámetros delta
    arcos = []
    c = {}
    delta = {}
    for k in K:
        for l in L:
            for t in T:
                for l_prime in L:
                    key = (l, l_prime)
                    tiempo_viaje = T_l_lprime.get(key, N+1)  # Movimiento inválido si no está definido
                    t_prime = t + tiempo_viaje
                    if t_prime <= N and tiempo_viaje > 0:
                        arco = (k, l, t, l_prime, t_prime)
                        arcos.append(arco)
                        # Asignar costos
                        if l != l_prime:
                            c[arco] = c_mov
                        else:
                            if l != 'P':
                                c[arco] = c_espera
                            else:
                                c[arco] = 0
                        # Asignar delta
                        if (l, l_prime) in [('C1', 'C2'), ('C2', 'C1')]:
                            if (l, l_prime) == ('C1', 'C2'):
                                delta[(arco, 1)] = 1
                                delta[(arco, 2)] = 0
                            else:
                                delta[(arco, 1)] = 0
                                delta[(arco, 2)] = 1
                        else:
                            delta[(arco, 1)] = 0
                            delta[(arco, 2)] = 0

    # Inicializar demanda para cada sentido y tiempo
    demanda = {}
    for s in S:
        for t in T:
            demanda[s, t] = D[s][t] if t < len(D[s]) else 0

    # Crear el modelo
    model = gp.Model(f"Despacho_de_Buses_{nombre_ruta}")

    # Variables de decisión
    x = model.addVars(arcos, vtype=GRB.BINARY, name="x")
    L_var = model.addVars(K, L, T, vtype=GRB.BINARY, name="L")
    y = model.addVars(S, T, vtype=GRB.CONTINUOUS, lb=0, name="y")
    p = model.addVars(K, S, T, vtype=GRB.CONTINUOUS, lb=0, name="p")
    z = model.addVars(K, T, vtype=GRB.BINARY, name="z")  # Bus en operación

    # Función objetivo ajustada para maximizar ingresos menos costos
    model.setObjective(
        c_ingreso * gp.quicksum(p[k, s, t] for k in K for s in S for t in T) -  # Ingresos por pasajeros transportados
        (gp.quicksum(c_bus * z[k, t] for k in K for t in T) +                # Costos por mantener el bus en operación
         gp.quicksum(c[arco] * x[arco] for arco in arcos)),                  # Costos de movimiento y espera
        GRB.MAXIMIZE
    )

    # ================================
    # 2. Definición de Restricciones
    # ================================

    # 1. Ubicación inicial de los buses
    for k in K:
        model.addConstr(L_var[k, 'P', 0] == 1, name=f"ubicacion_inicial_{k}")
        for l in L:
            if l != 'P':
                model.addConstr(L_var[k, l, 0] == 0, name=f"ubicacion_inicial_{k}_{l}")

    # 2. Exclusividad de ubicación
    for k in K:
        for t in T:
            model.addConstr(
                gp.quicksum(L_var[k, l, t] for l in L) <= 1,
                name=f"exclusividad_{k}_{t}"
            )

    # 3. Actualización de presencia de los buses
    for k in K:
        for l in L:
            for t in T[:-1]:
                salidas = gp.quicksum(
                    x[arco]
                    for arco in arcos
                    if arco[0] == k and arco[1] == l and arco[2] == t
                )
                llegadas = gp.quicksum(
                    x[arco]
                    for arco in arcos
                    if arco[0] == k and arco[3] == l and arco[4] == t + 1
                )
                model.addConstr(
                    L_var[k, l, t+1] == L_var[k, l, t] - salidas + llegadas,
                    name=f"actualizacion_{k}_{l}_{t}"
                )

    # 4. Movimiento solo si el bus está presente
    for arco in arcos:
        model.addConstr(
            x[arco] <= L_var[arco[0], arco[1], arco[2]],
            name=f"movimiento_{arco}"
        )

    # 5. Definición de z[k, t]
    for k in K:
        for t in T:
            model.addConstr(
                z[k, t] == 1 - L_var[k, 'P', t],
                name=f"bus_en_operacion_{k}_{t}"
            )

    # 6. Capacidad del bus en cada intervalo
    for k in K:
        for t in T:
            model.addConstr(
                gp.quicksum(p[k, s, t] for s in S) <= C * z[k, t],
                name=f"capacidad_{k}_{t}"
            )

    # 7. Satisfacción de la demanda
    for s in S:
        for t in T:
            model.addConstr(
                gp.quicksum(p[k, s, t] for k in K) + y[s, t] == demanda[s, t],
                name=f"demanda_{s}_{t}"
            )

    # 8. Consistencia entre movimiento y atención de pasajeros
    for k in K:
        for s in S:
            for t in T:
                # Pasajeros transportados solo si el bus está realizando un movimiento en ese sentido
                movimientos_s = gp.quicksum(
                    x[arco] * delta.get((arco, s), 0)
                    for arco in arcos
                    if arco[0] == k and arco[2] <= t < arco[4]
                )
                model.addConstr(
                    p[k, s, t] <= C * movimientos_s,
                    name=f"consistencia_{k}_{s}_{t}"
                )

    # 9. No exceder la capacidad del bus acumulada
    for k in K:
        for s in S:
            for t in T:
                pasajeros_acumulados = gp.quicksum(p[k, s, tau] for tau in T if tau <= t)
                viajes_s = gp.quicksum(
                    x[arco]
                    for arco in arcos
                    if arco[0] == k and delta.get((arco, s), 0) == 1 and arco[2] <= t
                )
                model.addConstr(
                    pasajeros_acumulados <= C * viajes_s,
                    name=f"capacidad_acumulada_{k}_{s}_{t}"
                )

    # ================================
    # 3. Parámetros de Optimización
    # ================================

    # Ajustar parámetros de Gurobi si se proporcionan
    # Configurar parámetros de Gurobi
    model.setParam('MIPGap', mipgap)                # Brecha de MIP
    model.setParam('TimeLimit', timelimit)          # Límite de tiempo
    model.setParam('MIPSepCuts', mipsepcuts)        # Cortes de separación de MIP
    model.setParam('ImpliedCuts', impliedcuts)      # Cortes implícitos
    model.setParam('NetworkCuts', networkcuts)      # Cortes de red
    model.setParam('FlowCoverCuts', flowcovercuts)  # Cortes de cobertura de flujo
    model.setParam('LiftProjectCuts', liftprojectcuts) # Cortes de lifting y proyección
    model.setParam('CliqueCuts', cliquecuts)        # Cortes de cliques

    # Optimizar el modelo
    model.optimize()

    # ================================
    # 4. Procesamiento de Resultados
    # ================================

    resultados = {}
    if model.status == GRB.OPTIMAL or model.status == GRB.TIME_LIMIT:
        resultados['estado'] = model.status
        resultados['obj_val'] = model.ObjVal
        resultados['runtime'] = model.Runtime
        resultados['gap'] = model.MIPGap
        resultados['pasajeros_atendidos'] = {}
        resultados['pasajeros_no_satisfechos'] = {}
        resultados['nivel_servicio'] = {}
        for s in S:
            demanda_total = sum(demanda[s, t] for t in T)
            atendida = demanda_total - sum(y[s, t].X for t in T)
            resultados['pasajeros_atendidos'][s] = atendida
            resultados['pasajeros_no_satisfechos'][s] = sum(y[s, t].X for t in T)
            resultados['nivel_servicio'][s] = (atendida / demanda_total) * 100 if demanda_total > 0 else 0
        resultados['nivel_servicio_global'] = sum(resultados['pasajeros_atendidos'].values()) / sum(
            demanda[s, t] for s in S for t in T
        ) * 100 if sum(demanda[s, t] for s in S for t in T) > 0 else 0

        # Métricas por bus
        resultados['metrics_bus'] = {}
        distancia_sentido = {1: 20, 2: 22}  # Distancias en km para cada sentido (pueden variar por ruta)
        for k in K:
            # Total de pasajeros transportados
            pasajeros_k = sum(p[k, s, t].X for s in S for t in T)

            # Total de viajes realizados (cualquier movimiento)
            viajes_k = sum(x[arco].X for arco in arcos if arco[0] == k and x[arco].X > 0.5)

            # Total de intervalos en operación
            intervalos_operacion = sum(z[k, t].X for t in T)

            # Total de intervalos realizando trayectos de recogida de pasajeros
            intervalos_recogida = 0
            for arco in arcos:
                if arco[0] == k and x[arco].X > 0.5:
                    sentido = None
                    for s in S:
                        if delta.get((arco, s), 0) == 1:
                            sentido = s
                            break
                    if sentido:
                        intervalos_recogida += T_l_lprime.get((arco[1], arco[3]), 0)

            # Utilización como porcentaje
            utilizacion_k = (intervalos_recogida * intervalo_minutos) / (intervalos_operacion * intervalo_minutos) * 100 if intervalos_operacion > 0 else 0

            # Kilómetros recorridos en trayectos de recogida de pasajeros
            km_k = 0
            for s in S:
                viajes_k_s = sum(
                    x[arco].X for arco in arcos
                    if arco[0] == k and delta.get((arco, s), 0) == 1 and x[arco].X > 0.5
                )
                km_k += viajes_k_s * distancia_sentido[s]
            ipk_k = pasajeros_k / km_k if km_k > 0 else 0

            resultados['metrics_bus'][k] = {
                'Pasajeros Transportados': pasajeros_k,
                'Viajes Realizados': viajes_k,
                'Utilización (%)': utilizacion_k,
                'Kilómetros Recorridos': km_k,
                'IPK (Pasajeros/km)': ipk_k
            }
    else:
        resultados['estado'] = model.status
        print(f"El modelo para la ruta {nombre_ruta} no encontró una solución óptima.")

    return resultados, model, x, L_var, p, y, z, arcos, delta, T_l_lprime, K, L, S, T, demanda


In [4]:
# ================================
# 3. Ejecución del Modelo para Todas las Rutas
# ================================

# Almacenar resultados de todas las rutas
resultados_totales = {}

# Lista de rutas a optimizar
lista_rutas = list(rutas.keys())

for ruta in lista_rutas:
    print(f"Optimizing route: {ruta}")
    resultados, modelo, x, L_var, p, y, z, arcos, delta, T_l_lprime, K, L, S, T, demanda = optimizar_ruta(ruta, rutas[ruta])
    
    resultados_totales[ruta] = {
        'resultados': resultados,
        'modelo': modelo,
        'variables': {
            'x': x,
            'L_var': L_var,
            'p': p,
            'y': y,
            'z': z
        },
        'arcos': arcos,
        'delta': delta,
        'T_l_lprime': T_l_lprime,
        'K': K,
        'L': L,
        'S': S,
        'T': T,
        'demanda': demanda
    }

    print(f"Completed optimization for route: {ruta}\n")


Optimizing route: PRADERA_27
Set parameter Username
Set parameter LicenseID to value 2587942


Academic license - for non-commercial use only - expires 2025-11-20
Set parameter MIPGap to value 0.005
Set parameter TimeLimit to value 3600
Set parameter MIPSepCuts to value 2
Set parameter ImpliedCuts to value 1
Set parameter NetworkCuts to value 1
Set parameter FlowCoverCuts to value 1
Set parameter LiftProjectCuts to value 2
Set parameter CliqueCuts to value 2
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-12450H, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 14010 rows, 10906 columns and 193842 nonzeros
Model fingerprint: 0x7855b2b9
Variable types: 1746 continuous, 9160 integer (9160 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Presolve removed 2792 rows and 2746 columns
Presolve time: 0.66s
Presol

In [5]:
# Function to present computational results
def presentar_resultados_computacionales(nombre_ruta, resultados):
    print(f"Resultados Computacionales para la Ruta: {nombre_ruta}")
    estado = resultados['estado']
    if estado == GRB.OPTIMAL:
        estado_opt = "Óptimo"
    elif estado == GRB.TIME_LIMIT:
        estado_opt = "Límite de tiempo alcanzado"
    else:
        estado_opt = f"Estado {estado}"
    print(f"  Estado de Optimización: {estado_opt}")
    print(f"  Tiempo de Ejecución: {resultados['runtime']:.2f} segundos")
    print(f"  Gap de MIP: {resultados['gap']*100:.2f}%")
    print(f"  Valor Óptimo: {resultados['obj_val']:.2f}")
    print("")

# Function to present global metrics
def presentar_metricas_globales(nombre_ruta, resultados):
    print(f"Métricas Globales para la Ruta: {nombre_ruta}")
    nivel_servicio_global = resultados['nivel_servicio_global']
    print(f"  Nivel de Servicio Global: {nivel_servicio_global:.2f}%")
    pasajeros_totales = sum(m['Pasajeros Transportados'] for m in resultados['metrics_bus'].values())
    viajes_totales = sum(m['Viajes Realizados'] for m in resultados['metrics_bus'].values())
    km_totales = sum(m['Kilómetros Recorridos'] for m in resultados['metrics_bus'].values())
    utilizacion_promedio = sum(m['Utilización (%)'] for m in resultados['metrics_bus'].values()) / len(resultados['metrics_bus'])
    ipk_global = pasajeros_totales / km_totales if km_totales > 0 else 0
    print(f"  Pasajeros Totales Transportados: {pasajeros_totales}")
    print(f"  Viajes Totales Realizados: {viajes_totales}")
    print(f"  Kilómetros Totales Recorridos: {km_totales:.2f} km")
    print(f"  Utilización Promedio de los Buses: {utilizacion_promedio:.2f}%")
    print(f"  IPK Global (Pasajeros/km): {ipk_global:.2f}")
    print("")

# Function to present metrics per bus
def presentar_metricas_por_bus(nombre_ruta, resultados):
    print(f"Métricas por Bus para la Ruta: {nombre_ruta}")
    for k, m in resultados['metrics_bus'].items():
        print(f"  Bus {k}:")
        print(f"    - Pasajeros Transportados: {m['Pasajeros Transportados']}")
        print(f"    - Viajes Realizados: {m['Viajes Realizados']}")
        print(f"    - Utilización: {m['Utilización (%)']:.2f}%")
        print(f"    - Kilómetros Recorridos: {m['Kilómetros Recorridos']:.2f} km")
        print(f"    - IPK (Pasajeros/km): {m['IPK (Pasajeros/km)']:.2f}")
    print("")

# Iterate over each route and present the results
for ruta in lista_rutas:
    resultados = resultados_totales[ruta]['resultados']
    presentar_resultados_computacionales(ruta, resultados)
    presentar_metricas_globales(ruta, resultados)
    presentar_metricas_por_bus(ruta, resultados)


Resultados Computacionales para la Ruta: PRADERA_27
  Estado de Optimización: Óptimo
  Tiempo de Ejecución: 76.79 segundos
  Gap de MIP: 0.50%
  Valor Óptimo: 10811.00

Métricas Globales para la Ruta: PRADERA_27
  Nivel de Servicio Global: 97.55%
  Pasajeros Totales Transportados: 8038.0
  Viajes Totales Realizados: 166.0
  Kilómetros Totales Recorridos: 3256.00 km
  Utilización Promedio de los Buses: 96.66%
  IPK Global (Pasajeros/km): 2.47

Métricas por Bus para la Ruta: PRADERA_27
  Bus 1:
    - Pasajeros Transportados: 1040.0
    - Viajes Realizados: 21.0
    - Utilización: 97.83%
    - Kilómetros Recorridos: 420.00 km
    - IPK (Pasajeros/km): 2.48
  Bus 2:
    - Pasajeros Transportados: 1040.0
    - Viajes Realizados: 21.0
    - Utilización: 96.77%
    - Kilómetros Recorridos: 420.00 km
    - IPK (Pasajeros/km): 2.48
  Bus 3:
    - Pasajeros Transportados: 936.0
    - Viajes Realizados: 20.0
    - Utilización: 96.43%
    - Kilómetros Recorridos: 378.00 km
    - IPK (Pasajeros/km)